chrome://dino/

In [1]:
from __future__ import division

In [2]:
from autopy import *
import gc
from PIL import Image, ImageOps
import numpy as np
import time
import random
from matplotlib import cm
import os.path

In [3]:
import win32gui, win32com.client
import re

class WindowMgr:
    """Encapsulates some calls to the winapi for window management"""
    def __init__ (self):
        """Constructor"""
        self._handle = None

    def find_window(self, class_name, window_name = None):
        """find a window by its class_name"""
        self._handle = win32gui.FindWindow(class_name, window_name)

    def _window_enum_callback(self, hwnd, wildcard):
        '''Pass to win32gui.EnumWindows() to check all the opened windows'''
        if re.match(wildcard, str(win32gui.GetWindowText(hwnd))) != None:
            self._handle = hwnd

    def find_window_wildcard(self, wildcard):
        self._handle = None
        win32gui.EnumWindows(self._window_enum_callback, wildcard)
        
   
    def find_window_position_size(self):
        rect = win32gui.GetWindowRect(self._handle)
        x = rect[0]
        y = rect[1]
        w = rect[2] - x
        h = rect[3] - y
        return (x , y , w , h)
    
    def set_foreground(self):
        """put the window in the foreground"""
        shell = win32com.client.Dispatch("WScript.Shell")
        shell.SendKeys('%')
        win32gui.SetForegroundWindow(self._handle)
        
    def get_screen_image(self):
        x , y , w , h = self.find_window_position_size()
        bmp = bitmap.capture_screen()
        bmp.save('screen.png')
        img = Image.open('screen.png')
        img = img.crop((x + 10 , y + 150  , x+w -10 , y+ h - 200 ))
        img = img.resize((256,256)).convert('L')
        img = ImageOps.invert(img)
        shape = [(0, 55), (w, 50)]
        img = np.asarray(img)
        img = img / 255.0
        img = img.reshape((256,256))
        return img

In [4]:
def is_done(image):
    avg = np.average(image[52:55 , 0:255])
    if avg < 0.05:
        return True 
    else:
        return False

In [5]:
def key_press(code):
        key.toggle(code , True)
        key.toggle(code , False)

In [6]:
def key_tap(code):
    key.tap(code , delay = 0.00000001)
#     key.toggle(code , True)
#     time.sleep(0.000001)
#     key.toggle(code , False)

In [7]:
class Gym:
    def __init__(self):
        self.window = None
        
    def reset(self):
        self.window = WindowMgr()
        self.window.find_window_wildcard(".*chrome://dino/.*")
        self.window.set_foreground()
        key_press(key.Code.F5)
        time.sleep(0.08)
        key_press(key.Code.SPACE)
        img = np.zeros((256,256,3))
        return img
    
    def step(self , action , delay):
        
        
        if action == 2 :
            key_press(key.Code.UP_ARROW)
        if action == 0 :
            key_tap(key.Code.UP_ARROW)
        if action == 3 :
            key_press(key.Code.DOWN_ARROW)
        if action == 1 :
            pass
#         delay = 0 if 0.1 - delay <=0 else 0.1 - delay
#         time.sleep(delay)
        img1 = self.window.get_screen_image()
        time.sleep(1/50)
        img2 = self.window.get_screen_image()
        time.sleep(1/50)
        img3 = self.window.get_screen_image()
        
        img = np.stack([img1 , img2 , img3] , axis=-1)

        done = is_done(img3)


        if done : 
            reward =  +1
        else:
            reward = -1
  
        return img , done , reward

In [8]:
env = Gym()
# env.reset()
# done = True
# while done :
#     observation,done,reward = env.step(np.random.randint(0,3))
# #     im = Image.fromarray(np.uint8(cm.gist_earth(observation)*255))
# #     display(im)

In [9]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [10]:
class iDQNAgent:
    def __init__(self, state_shape , action_no, memory_size = 1500):
        self.action_no = action_no
        self.memory = []
        self.memory_size = memory_size
        self.gamma = 0.9
        self.epsilon = 1.0
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.999
        self.learning_rate = 1e-4
        self.sync_nets = 50

        
        self.state_shape = state_shape

        
        self.q_model = self.build_model(state_shape , action_no)
        self.q_model.compile(loss='mse' , optimizer=Adam(learning_rate=self.learning_rate))
        
#         if os.path.exists('q_model.h5'):
#             self.q_model.load_weights('q_model.h5')
            
        self.target_q_model = self.build_model(state_shape , action_no)
        
        self.update_weights()
    
        
        ####################################################
    def build_model(self ,input_shape , n_outputs):
        inputs = layers.Input(shape = input_shape , name='state')
        x = layers.Conv2D(64, (4,4) , strides=4 ,padding='same' , kernel_initializer='he_normal')(inputs)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.Conv2D(64, (4,4) , strides=2 ,padding='same' , kernel_initializer='he_normal')(x)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.Conv2D(64, (3,3) , strides=2 ,padding='same' , kernel_initializer='he_normal')(x)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
#         x = layers.MaxPooling2D()(x)
        
        x = layers.Conv2D(64, (3,3) , strides=2 ,padding='same' , kernel_initializer='he_normal')(x)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
#         x = layers.MaxPooling2D()(x)
        
        x = layers.Conv2D(64, (3,3) , strides=2 ,padding='same' , kernel_initializer='he_normal')(x)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
#         x = layers.MaxPooling2D()(x)
   
        x = layers.Flatten()(x) 

        x = layers.Dense(512 , activation='relu')(x)
        x = layers.Dense(256 , activation='relu')(x)
        x = layers.Dense(n_outputs , activation='linear' , name = 'action')(x)
        model = Model(inputs , x)
#         model.summary()
        return model
    
        #####################################################
    def act(self , state):
        
        if np.random.rand() < self.epsilon:
            return np.random.randint(0 , self.action_no) , 0
            
        t = time.time()
        q_values = self.q_model.predict(np.expand_dims(state , axis=0))
        action = np.argmax(q_values[0])
        print("action {} q-value {} ".format(action , q_values[0][action] ))
        return action , time.time()-t
    
        ######################################################
    def remember(self, state, action, reward , next_state , done):
        item = (state , action , reward , next_state ,done)
        if len(self.memory) > self.memory_size :
            self.memory.pop(0)
        self.memory.append(item)
        
        #######################################################
    def get_target_q_value(self, next_state, reward , double):
        if double :
            action = np.argmax(self.q_model.predict(next_state[0]))
            q_value = self.target_q_model.predict(next_state)[0][action]
        else :
            q_value = np.amax(self.target_q_model.predict(next_state)[0])
        
        q_value *= self.gamma
        q_value += reward
        return q_value
    
        ########################################################
    def replay(self , batch_size):
        sars_batch = random.sample(self.memory , batch_size)
       
        state_batch , q_value_batch = [] , []
        for state, action, reward, next_state, done in sars_batch:
            
            q_values = self.q_model.predict(np.expand_dims(state , axis=0))
            
            q_value = self.get_target_q_value(np.expand_dims(next_state , axis=0), reward, False)
            # ??????????
            q_values[0][action] = reward if done==False else q_value
            
            state_batch.append(state)
            q_value_batch.append(q_values[0])
        
#         print('q-values mean: {}'.format(np.mean(q_value_batch)))
        
        hist = self.q_model.fit(np.array(state_batch) , 
                             np.array(q_value_batch),
                             batch_size = batch_size,
                            epochs = 1,
                            verbose = 0)
                
        return hist.history['loss'][0]
       
       ######################################################

       ######################################################
    def update_epsilon(self ):
#         self.epsilon = self.epsilon_min + (1 - self.epsilon_min)* np.exp(-self.epsilon_decay * episode)
        self.epsilon = self.epsilon  * self.epsilon_decay
        if self.epsilon < self.epsilon_min :
            self.epsilon = self.epsilon_min

        ########################################################
    def update_weights(self):
            self.target_q_model.set_weights(self.q_model.get_weights())


In [11]:
episode_count = 1000
time_steps =    100
batch_size = 32

In [12]:
agent = iDQNAgent((256 , 256 , 3) ,4)

In [17]:
all_rewards = []
all_losses =  []
loss = 0

for episode in range(episode_count):
    state = env.reset()
    done = True
    total_reward = 0

    for t in range(time_steps) :
        action , delay = agent.act(state)
        next_state , done ,reward = env.step(action , delay)
#         if not done:
#             reward -= 100
        agent.remember(state , action , reward , next_state , done)
        state = next_state
        total_reward += reward
   
        if not done : 
            break
            
    if len(agent.memory) >= batch_size:
        loss = agent.replay(batch_size)
        all_losses.append(loss) 
        
    agent.update_epsilon()
    
    if episode % agent.sync_nets == 0:
        agent.update_weights()
        print('update weights')
        print("Avarage loss {}".format(np.mean(np.array(all_losses[-50:]))) )
        print("Avarage reward {}".format(np.mean(np.array(all_rewards[-50:]))) )
        
    if episode % 100 == 0:
        agent.q_model.save_weights('q_model.h5')
        
    print("episode {} reward {} loss {} epsilon {} memory {}".format(episode , total_reward, loss, agent.epsilon , len(agent.memory)))
    all_rewards.append(total_reward)
    
    if loss > 2 :
        break

action 0 q-value 5.592217445373535 
action 0 q-value 6.84645414352417 
action 0 q-value 6.5044846534729 
action 0 q-value 5.841061592102051 
action 0 q-value 6.384753704071045 
action 2 q-value 5.9116129875183105 
action 0 q-value 5.815375328063965 
action 0 q-value 6.1526994705200195 
action 0 q-value 4.627495288848877 
action 0 q-value 5.373902320861816 
action 1 q-value 4.228018760681152 
action 0 q-value 5.865904808044434 
action 0 q-value 4.820282936096191 
action 1 q-value 5.071661949157715 
update weights
Avarage loss 0.20119881629943848
Avarage reward nan
episode 0 reward 22 loss 0.20119881629943848 epsilon 0.36732772934619257 memory 1501


C:\Users\SERVER\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\SERVER\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


action 2 q-value 6.063782215118408 
action 2 q-value 5.679942607879639 
action 0 q-value 5.8054986000061035 
action 3 q-value 5.724363327026367 
action 0 q-value 6.36414909362793 
action 3 q-value 5.373510837554932 
action 0 q-value 6.189960479736328 
action 0 q-value 5.488467693328857 
action 1 q-value 4.698291301727295 
episode 1 reward 14 loss 0.4306996762752533 epsilon 0.3669604016168464 memory 1501
action 0 q-value 6.293349266052246 
action 2 q-value 6.043045520782471 
action 3 q-value 5.732719898223877 
action 0 q-value 5.417675018310547 
action 0 q-value 5.607264041900635 
action 0 q-value 6.596777439117432 
action 2 q-value 6.160391330718994 
action 0 q-value 6.675529956817627 
action 2 q-value 6.2659430503845215 
action 0 q-value 6.1381707191467285 
action 0 q-value 5.788549900054932 
action 1 q-value 4.233885765075684 
action 0 q-value 6.039529800415039 
action 1 q-value 4.635814189910889 
action 0 q-value 5.633655071258545 
action 0 q-value 6.598936557769775 
action 1 q-valu

action 1 q-value 5.6365227699279785 
episode 14 reward 14 loss 0.1677362024784088 epsilon 0.3622184346183837 memory 1501
action 0 q-value 6.440513610839844 
action 1 q-value 6.168646812438965 
action 1 q-value 5.862366199493408 
action 0 q-value 5.988455295562744 
action 3 q-value 7.038169860839844 
action 0 q-value 6.237118244171143 
action 0 q-value 4.809916019439697 
action 3 q-value 6.234857082366943 
action 0 q-value 4.343008041381836 
action 3 q-value 5.481147289276123 
action 3 q-value 5.09028959274292 
episode 15 reward 17 loss 0.24135170876979828 epsilon 0.36185621618376534 memory 1501
action 0 q-value 6.42282772064209 
action 1 q-value 6.017248630523682 
action 1 q-value 6.070249080657959 
action 0 q-value 5.946129322052002 
action 3 q-value 7.108558177947998 
action 0 q-value 5.982288360595703 
action 3 q-value 6.845065116882324 
action 0 q-value 6.46591854095459 
action 3 q-value 6.122785568237305 
action 0 q-value 5.844986915588379 
action 3 q-value 6.365138053894043 
acti

action 2 q-value 4.944275379180908 
action 1 q-value 6.444953918457031 
action 2 q-value 5.640158176422119 
action 1 q-value 6.328149318695068 
action 0 q-value 4.9701828956604 
action 3 q-value 6.0029683113098145 
action 1 q-value 5.600820064544678 
action 3 q-value 5.496335983276367 
action 0 q-value 6.572843551635742 
action 1 q-value 5.33140230178833 
action 3 q-value 1.7600061893463135 
action 3 q-value 2.4414985179901123 
episode 28 reward 35 loss 0.3086916506290436 epsilon 0.3571802069256231 memory 1501
action 0 q-value 6.729824542999268 
action 1 q-value 6.150057792663574 
action 1 q-value 6.1246466636657715 
action 0 q-value 6.056060314178467 
action 0 q-value 6.629514217376709 
action 3 q-value 7.661246299743652 
action 0 q-value 7.445796489715576 
action 0 q-value 5.81614351272583 
action 3 q-value 6.430532932281494 
action 3 q-value 4.434590816497803 
action 0 q-value 2.8489842414855957 
episode 29 reward 14 loss 0.30805814266204834 epsilon 0.3568230267186975 memory 1501
ac

...

action 0 q-value 7.736420154571533 
action 2 q-value 9.60421085357666 
action 1 q-value 5.916847229003906 
action 2 q-value 8.546529769897461 
action 1 q-value 1.9601759910583496 
action 3 q-value 0.932311475276947 
episode 152 reward 29 loss 0.9625480771064758 epsilon 0.3155062193168902 memory 1501
action 0 q-value 7.023285865783691 
action 2 q-value 8.704071044921875 
action 2 q-value 8.562472343444824 
action 2 q-value 9.022607803344727 
action 2 q-value 9.155012130737305 
action 2 q-value 8.366829872131348 
action 3 q-value 0.0758650153875351 
episode 153 reward 15 loss 0.5144145488739014 epsilon 0.31519071309757335 memory 1501
action 0 q-value 7.0986738204956055 
action 0 q-value 8.12963581085205 
action 0 q-value 7.491408824920654 
action 2 q-value 8.349649429321289 
action 2 q-value 9.183905601501465 
action 2 q-value 8.619479179382324 
action 2 q-value 8.648086547851562 
action 2 q-value 7.447925567626953 
episode 154 reward 14 loss 0.37514591217041016 epsilon 0.314875522384475

action 1 q-value 6.836897373199463 
action 0 q-value 0.38563624024391174 
episode 165 reward 41 loss 0.1641194224357605 epsilon 0.31142915794128007 memory 1501
action 0 q-value 8.247367858886719 
action 0 q-value 8.456581115722656 
action 0 q-value 8.849644660949707 
action 0 q-value 8.041810989379883 
action 3 q-value 7.969892978668213 
action 3 q-value 8.868776321411133 
action 3 q-value 8.033467292785645 
action 3 q-value 7.955228805541992 
action 0 q-value 7.496088981628418 
action 3 q-value 6.042357444763184 
action 1 q-value 6.418790340423584 
action 0 q-value 7.07218074798584 
action 3 q-value 2.6182515621185303 
episode 166 reward 16 loss 0.465649276971817 epsilon 0.31111772878333877 memory 1501
action 0 q-value 8.228658676147461 
action 0 q-value 8.18642520904541 
action 0 q-value 8.203670501708984 
action 0 q-value 8.529877662658691 
action 0 q-value 8.468525886535645 
action 3 q-value 7.957299709320068 
action 3 q-value 8.412638664245605 
action 3 q-value 7.989990234375 
act

action 2 q-value 5.905505657196045 
action 0 q-value 6.832505226135254 
action 1 q-value 4.409250736236572 
action 2 q-value -1.129431962966919 
episode 177 reward 20 loss 0.960698127746582 epsilon 0.3077124940099052 memory 1501
action 0 q-value 8.736129760742188 
action 3 q-value 8.7450532913208 
action 3 q-value 8.778660774230957 
action 3 q-value 8.864043235778809 
action 0 q-value 9.229607582092285 
action 3 q-value 10.504335403442383 
action 3 q-value 10.472967147827148 
action 3 q-value 9.999594688415527 
action 3 q-value 10.254854202270508 
action 3 q-value 9.472733497619629 
action 2 q-value 7.79349422454834 
action 0 q-value 7.308187007904053 
action 3 q-value 5.924025535583496 
action 0 q-value 7.740310192108154 
action 3 q-value 8.03372573852539 
action 3 q-value 8.006489753723145 
action 0 q-value 5.873729228973389 
action 1 q-value 6.4618239402771 
action 0 q-value 7.5836381912231445 
action 3 q-value 9.4473295211792 
action 3 q-value 7.451765060424805 
action 2 q-value 8.

action 0 q-value 8.69499683380127 
action 0 q-value 7.388037204742432 
action 1 q-value 9.12341022491455 
action 2 q-value 6.881103038787842 
action 1 q-value 7.791723251342773 
action 2 q-value 8.121744155883789 
action 1 q-value 7.856816291809082 
action 2 q-value 10.518726348876953 
action 1 q-value 6.277789115905762 
action 3 q-value 2.7251224517822266 
action 1 q-value 7.097609996795654 
action 2 q-value 7.833808422088623 
action 1 q-value 8.814946174621582 
action 0 q-value 10.965033531188965 
action 2 q-value 6.492266654968262 
action 1 q-value 6.6594719886779785 
action 0 q-value 10.464519500732422 
action 1 q-value 5.6890339851379395 
action 0 q-value 7.6553120613098145 
action 1 q-value 7.577315807342529 
action 2 q-value 5.411564826965332 
action 1 q-value 6.942942142486572 
action 0 q-value 7.394148349761963 
episode 190 reward 38 loss 0.4340824484825134 epsilon 0.30373614537615473 memory 1501
action 0 q-value 7.765657424926758 
action 1 q-value 7.975968837738037 
action 1 

action 2 q-value 5.0851240158081055 
action 3 q-value 6.094799041748047 
action 2 q-value 1.7907507419586182 
episode 201 reward 21 loss 1.2599155902862549 epsilon 0.30041170324864147 memory 1501
action 0 q-value 7.717287063598633 
action 3 q-value 8.171815872192383 
action 3 q-value 9.5398588180542 
action 0 q-value 8.257655143737793 
action 3 q-value 9.18929672241211 
action 0 q-value 8.471090316772461 
action 3 q-value 9.273308753967285 
action 0 q-value 7.000768661499023 
action 3 q-value 7.4676384925842285 
action 2 q-value 2.469423532485962 
action 3 q-value 2.4412784576416016 
action 2 q-value 0.16312502324581146 
episode 202 reward 16 loss 1.6136473417282104 epsilon 0.30011129154539284 memory 1501
action 0 q-value 7.719265460968018 
action 3 q-value 8.06010913848877 
action 3 q-value 8.132822036743164 
action 1 q-value 8.337331771850586 
action 0 q-value 8.012694358825684 
action 0 q-value 7.766616344451904 
action 0 q-value 7.363729000091553 
action 3 q-value 9.653592109680176

In [23]:
state = env.reset()
done = True
while done :
    q_values = agent.q_model.predict(np.expand_dims(state , axis=0))
    action = np.argmax(q_values[0])
    next_state , done ,reward  = env.step(action ,0)
    state = next_state
#     im = Image.fromarray(np.uint8(cm.gist_earth(observation)*255))
#     display(im)